## Ανάλυση Κανόνων Συσχετίσεων
* Σε αυτό το αρχείο θα αναζητήσω βιβλία που αγοράστηκαν συνδυαστικά και εκτιμήθηκαν
* Έτσι, θα μπορέσουμε να προτείνουμε βιβλία σε άλλους χρήστες που πιθανόν να τους αρέσουν
* Δηλώνω τις βιβλιοθήκες που θα χρησιμοποιήσω

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

* fact_table.txt -> το αρχείο έχει εξαχθεί από το data warehouse και αντιστοιχεί στον πίνακα fact Table όπως προέκυψε μετά τη διαδικασία ETL

In [2]:
data = pd.read_csv('fact_table.txt', sep=",")

* Με δεδομένο ότι πλέον κάνω κανόνα συσχέτισης μόνο για τα "ωραία βιβλία" αφαιρώ βιβλία με κακές αξιολογήσεις και συγκεκριμένα με αξιολογήσεις μικρότερες του 6.
* Με άλλα λόγια, πλέον δεν με ενδιαφέρει μόνο τα βιβλία που αγοράστηκαν συνδυαστηκάν αλλά τα βιβλία που αγοράστηκαν συνδυαστικά, απέκτησαν υψηλές αξιολογήσεις και είχαν απήχηση στους χρήστες

In [3]:
data_high = data[data['Rating_Score']>= 6].copy()

* Διώχνω όποιο βιβλίο έχει αγοραστεί λιγότερο από 100 φορές

In [4]:
new = data_high.groupby('Book').filter(lambda x : len(x)>50)

* Κρατάω μόνο όσους χρήστες έχουν αγοράσει πάνω από τρεις φορές ώστε να έχει νόημα η πρόταση βιλίων 

In [5]:
final = new.groupby('Reader').filter(lambda x : len(x)>3).copy()

* Διατηρώ τους εναπομείναντες μοναδικούς τίτλους βιβλίων

In [6]:
titles = pd.DataFrame (final['Book'].unique().copy(), columns = ['Book_Title'])

* Διατηρώ τους εναπομείναντες μοναδικούς χρήστες

In [7]:
users = final['Reader'].unique().copy()

In [8]:
len(users)

1114

* Φτιάχνω ένα dictionary για τις ανάγκες του πίνακα. Δηλαδή ξεκινώντας από το 1 εως τον αριθμό του κάθε χρήστη και αντιστοιχώ user_id κατά αύξοντα αριθμό

In [9]:
users_dict = {}
index = 0
for i in users:
    users_dict[i] = index
    index = index + 1
len(users_dict)

1114

* Φτιάχνω ένα data frame με γραμμές όσους και χρήστες και στήλες όσα και τα βιβλία. Γεμίζω το data frame με τιμές False

In [10]:
association_df = pd.DataFrame(False, index=np.arange(1114), columns=titles.Book_Title)

* Από το πίνακα final όπου περιέχει τα τελικά δεδομένα του Fact Table έτσι όπως εξήχθησαν από το Warehouse, ελέγχω τα βιβλία που διάβασε ο κάθε χρήστης. Με άλλα λόγια ελέγχω τον Fact Table και μόλις βρω ότι ο χρήστης έκανε κάποια κριτική για κάποιο βιβλίο πηγαίνω και σημειώνω στο association_df ότι ο συγκεκριμένος χρήστης όπου βρίσκεται στη γραμμή users_dict[row['Reader'] έχει διαβάσει το βιβλίο που βρίσκεται στη στήλη row['Book_Title']. Με αυτό το τρόπο έχω στο association_df την πληροφορία ποιος χρήστης διάβασε ποια βιβλία.

In [11]:
for index, row in final.iterrows():
  association_df.at[users_dict[row['Reader']], row['Book']] = True

* Από προεπιλογή, το apriori επιστρέφει τους δείκτες των στηλών των στοιχείων, οι οποίοι μπορεί να είναι χρήσιμοι σε επόμενες λειτουργίες, όπως η εξόρυξη κανόνων συσχέτισης. Για καλύτερη αναγνωσιμότητα, μπορούμε να ορίσουμε use_colnames=True για να μετατρέψουμε αυτές τις ακέραιες τιμές στα αντίστοιχα ονόματα στοιχείων

In [12]:
df = apriori(association_df, min_support=0.005, use_colnames=True, verbose=1, max_len= None, low_memory=False )

Processing 264 combinations | Sampling itemset size 6543


* Επιλέγω τους κορυφαίους 10 συνδυασμούς

In [13]:
df_ar = association_rules(df, metric = "confidence")
final_results = df_ar.sort_values(by=['confidence'], ascending=False).iloc[:5][['antecedents', 'consequents', 'support', 'confidence', 'lift']]

* Εμφανίζω τους κωδικούς βιβλίων των τελικών αποτελεσμάτων. Απομένει να αντικαταστήσω τους κωδικούς με τους τίτλους των βιβλίων

In [14]:
final_results

,antecedents,consequents,support,confidence,lift
233,"(36067, 36422, 36835)","(36120, 36277)",0.006284,1.0,42.846154
158,"(36067, 130572, 36422)",(36120),0.010772,1.0,25.906977
90,"(82995, 123557, 38215)",(82757),0.006284,1.0,13.925000
154,"(8585, 36067, 36422)",(36120),0.005386,1.0,25.906977
87,"(82995, 123557, 38215)",(82974),0.006284,1.0,12.804598


* Για να εμφανίσω τους τίτλους βιβλίων χρειάζομαι το πίνακα dimension book_title_dim

In [16]:
book_dim = pd.read_csv('bookdimension.csv', sep="|")

* Μετανομάζω και αφαιρώ κάποιες αχρείαστες στήλες

In [17]:
book_dim.columns = book_dim.columns.str.replace('book_id', 'Book_Title')
book_dim.columns = book_dim.columns.str.replace('book_title', 'book_title_label')
book_dim = book_dim.drop(['ISBN', 'year_of_publication'], axis=1)

* Κάνω merge με τα βιβλία που χρησιμοποιήθηακν στον αλγόριθμο

In [18]:
book_results = pd.merge(book_dim, titles, on=['Book_Title'])

* Και εμφανίζω τα τελικά αποτελέσματα

In [19]:
for index, row in final_results.iterrows():
  asc =[]
  for r in row['antecedents']:
    for ind, rr in book_results.iterrows():
      if (r == rr['Book_Title']):
        asc.append(rr['book_title_label'])
        continue
  con =[]
  for r in row['consequents']:
    for ind, rr in book_results.iterrows():
      if (r == rr['Book_Title']):
        con.append(rr['book_title_label'])
        continue
  print(asc, con)
  print("----------->  " + 'support ' + str(row['support']) + " & confidence "+ str(row['confidence'])+ ' ')

['Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel)', 'Hot Six : A Stephanie Plum Novel (A Stephanie Plum Novel)', 'One for the Money (A Stephanie Plum Novel)'] ['Four To Score (A Stephanie Plum Novel)', 'High Five (A Stephanie Plum Novel)']
----------->  support 0.0062836624775583485 & confidence 1.0 
['Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel)', 'Two for the Dough', 'Hot Six : A Stephanie Plum Novel (A Stephanie Plum Novel)'] ['Four To Score (A Stephanie Plum Novel)']
----------->  support 0.010771992818671455 & confidence 1.0 
['Harry Potter and the Goblet of Fire (Book 4)', "Harry Potter and the Sorcerer's Stone (Book 1)", 'The Catcher in the Rye'] ['Harry Potter and the Chamber of Secrets (Book 2)']
----------->  support 0.0062836624775583485 & confidence 1.0 
['One for the Money (Stephanie Plum Novels (Paperback))', 'Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel)', 'Hot Six : A Stephanie Plum Novel (A Stephani